# Подготовка среды

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pickle as pc
import numpy as np
import pandas as pd

In [ ]:
import os

os.chdir('/content/drive/Shared drives/hse_BERT/hse_Af_Tr_BERT')

In [ ]:
!pip install transformers

     |████████████████████████████████| 2.1MB 9.8MB/s 
     |████████████████████████████████| 870kB 37.8MB/s 
     |████████████████████████████████| 3.3MB 35.9MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.44-cp37-none-any.whl size=886084 sha256=7eb1c896dc9c511369aef184268e2dce7e8c462e7e727e1601a7cb830a3d39ba
  Stored in directory: /root/.cache/pip/wheels/3e/fb/c0/13ab4d63d537658f448366744654323077c4d90069b6512f3c
Successfully built sacremoses


In [ ]:
!pip install razdel

In [ ]:
import torch
import nltk
import re
import json
import time
import numpy as np
from transformers import BertTokenizer, BertModel

# OPTIONAL: if you want to have more information on what's happening, activate the logger as follows
import logging
#logging.basicConfig(level=logging.INFO)

import matplotlib.pyplot as plt
% matplotlib inline

from scipy.spatial.distance import cosine

#all_emc


In [ ]:
# files = os.listdir("./embeddings/sent2/ready")
# res = dict()
# for i in files:
#     with open("./embeddings/sent2/ready/{}".format(i), "rb") as f:
#         temp = pc.load(f)
#         # print(type(temp))
#         res.update(temp)

In [ ]:
# with open("/content/drive/Shareddrives/hse_BERT/hse_Af_Tr_BERT/embeddings/sent2/all_emc.pic", "bw") as f:
#     pc.dump(res, f)

# read data

In [ ]:
!ls

bin		    hse_BERT_2.ipynb	models		  res_df2_avg.csv
embeddings	    hse_BERT_3_0.ipynb	mvenv		  res_df_2_mul.csv
hse_BERT_2_1.ipynb  hse_BERT_3_1.ipynb	README.md	  res_df2_mul.csv
hse_BERT_2_2.ipynb  hse_BERT_3_2.ipynb	res_df1_avg.csv   tests
hse_BERT_2_3.ipynb  hse_BERT_4_0.ipynb	res_df_1_mul.csv  texts
hse_BERT_2_4.ipynb  hse_BERT.ipynb	res_df1_mul.csv   venv


In [ ]:
with open("/content/drive/Shareddrives/hse_BERT/hse_Af_Tr_BERT/embeddings/sent1/all_emc.pic", "br") as f:
    dct1  = pc.load(f)
with open("/content/drive/Shareddrives/hse_BERT/hse_Af_Tr_BERT/embeddings/sent2/all_emc.pic", "br") as f:
    dct2  = pc.load(f)

In [ ]:
# dct.keys()

In [ ]:
len(dct2)

36453

In [ ]:
def get_word(emb, dct, nin = []):
    word = "none"
    cs = -100 
    for key in dct:
        temp_cs = 1 - cosine(emb, dct[key])
        if temp_cs > cs and key not in nin:
            cs = temp_cs
            word = key
    return word, cs

def get_word_m(emb, dct, count = 3):
    res = {}
    for key in dct:
        res[key] = 1 - cosine(emb, dct[key])
    l = list(res.items()) 
    l.sort(key= lambda x:x[1], reverse = True)
    return dict(l[:count])


In [ ]:
def add_names(df):
    names = {': capital-common-countries' : "Capitals-common-countries",
 ': capital-world': "Capital-countries",
 ': currency': "Currency",
#  ': city-in-state',
 ': family': "Family",
 ': gram1-adjective-to-adverb': "Adjective adverb",
 ': gram2-opposite': "Opposite",
 ': gram3-comparative': "Comparative",
#  ': gram4-superlative',
#  ': gram5-present-participle',
 ': gram6-nationality-adjective': "Nationality adjective",
#  ': gram7-past-tense',
#  ': gram8-plural',
#  ': gram9-plural-verbs'
    }
    df["type"] = "none"
    for i in names:
        if i in df.index:
            df.loc[i, "type"] = names[i]

    return df

def add_names_rus(df):
    names = {': capital-common-countries' : "Известные столицы - страны",
 ': capital-world': "Столицы - страны",
 ': currency': "Валюта",
#  ': city-in-state',
 ': family': "Семья",
 ': gram1-adjective-to-adverb': "Прилагательное - наречие",
 ': gram2-opposite': "Антонимы",
 ': gram3-comparative': "Сравнение",
#  ': gram4-superlative',
#  ': gram5-present-participle',
 ': gram6-nationality-adjective': "Национальность - прилагательное",
#  ': gram7-past-tense',
#  ': gram8-plural',
#  ': gram9-plural-verbs'
    }
    df["type"] = "none"
    for i in names:
        if i in df.index:
            df.loc[i, "type"] = names[i]

    return df

def add_names_rus_to_eng(df):
    names = { "Известные столицы - страны" : "Capitals-common-countries",
  "Столицы - страны": "Capital-countries",
  "Валюта": "Currency",
#  ': city-in-state',
  "Семья": "Family",
  "Прилагательное - наречие": "Adjective adverb",
  "Антонимы": "Opposite",
  "Сравнение": "Comparative",
#  ': gram4-superlative',
#  ': gram5-present-participle',
  "Национальность - прилагательное": "Nationality adjective",
#  ': gram7-past-tense',
#  ': gram8-plural',
#  ': gram9-plural-verbs'
    }
    df["type"] = "none"
    for i in names:
        if i in df.index:
            df.loc[i, "type"] = names[i]

    return df

In [ ]:
get_word_m(dct1["машина"], dct1, 10)

{'авто': 0.8991258144378662,
 'автомобиль': 0.9381362199783325,
 'автомобиля': 0.904218316078186,
 'машин': 0.9311237931251526,
 'машина': 1.0,
 'машинами': 0.9071072340011597,
 'машине': 0.9445964694023132,
 'машиной': 0.9558458924293518,
 'машину': 0.951924741268158,
 'машины': 0.975075364112854}

# GATS

In [ ]:
with open("/content/drive/Shareddrives/hse_BERT/hse_Af_Tr_BERT/tests/questions-words.txt", "r") as f:
    lines = f.read().splitlines()

words = {}
words2 = {}
all_w = []
key = "no_key"
for i in lines:
    m = i.split(" ")
    if i[0] == ":":
        key = i
    else:
        if key not in words:
            words[key] = []
            words2[key] = []
        words[key].append(m)
        all_w = all_w + m
        words2[key].append((m[0], m[1]))
        words2[key].append((m[2], m[3]))
all_w = set(all_w)
for key in words2:
    words2[key] = set(words2[key])

In [ ]:
# len(words2)

In [ ]:
# list(words2.keys())

In [ ]:
# gats = pd.DataFrame(all_w)
# gats.to_excel("tests/GATS.xlsx", index = None)

In [ ]:
# df2 = pd.read_excel("tests/GATS_rus.xlsx")
# gats["eng"] = gats[0]
# gats["rus"] = df2[0].apply(lambda x: x.lower())
# gats.index = gats["eng"]
# gats = gats[[ "rus"]]
# gats.head()

In [ ]:

# words2_rus = {}
# def translate(m):
#     m2 = []
#     for i in m:
#         m2.append(gats.loc[i]['rus'])
#     return m2
# for key in words:
#     # words_rus[key] = list(map(translate, words[key]))
#     words2_rus[key] = list(map(translate, words2[key]))


In [ ]:
# for key in words2_rus:
#     df = pd.DataFrame()
#     df["eng_w1"] = list(zip(*words2[key]))[0]
#     df["eng_w2"] = list(zip(*words2[key]))[1]
#     df["rus_w1"] = list(zip(*words2_rus[key]))[0]
#     df["rus_w2"] = list(zip(*words2_rus[key]))[1]
#     df.to_csv("tests/gats/{}.csv".format(key[2:]), index = None, sep = ";", encoding = "cp1251")

In [ ]:
names = [
        #  ': capital-common-countries',
 ': capital-world',
#  ': currency',
#  ': city-in-state',
 ': family',
 ': gram1-adjective-to-adverb',
 ': gram2-opposite',
 ': gram3-comparative',
#  ': gram4-superlative',
#  ': gram5-present-participle',
 ': gram6-nationality-adjective',
#  ': gram7-past-tense',
#  ': gram8-plural',
#  ': gram9-plural-verbs'
 ]

In [ ]:
words3 = {}
find_words = []
for key in names:
    df = pd.read_csv("tests/gats/{}.csv".format(key[2:]),sep = ";", encoding = "cp1251")
    d1 = list(zip(df["eng_w1"], df["rus_w1"]))
    d2 = list(zip(df["eng_w2"], df["rus_w2"]))
    words3[key] = dict(d1)
    words3[key].update(dict(d2))
    all_rus = [*df["rus_w1"], *df["rus_w2"]]

In [ ]:
# eng = list(zip(df["eng_w1"], df["eng_w2"]))
# rus = list(zip(df["rus_w1"], df["rus_w2"]))
pd.DataFrame(all_rus).to_csv("tests/gats/all_rus.csv", index = None)

In [ ]:
def translate_sl(m ,sl):
    m2 = []
    for i in m:
        m2.append(sl[i])
    return m2

In [ ]:
words_rus = {}
for key in names:
    words_rus[key] = []
    for i in words[key]:
        words_rus[key].append(translate_sl(i ,words3[key]))

In [ ]:
with open("tests/gats/ready.pic", "bw") as f:
    pc.dump(words_rus, f)

In [ ]:
with open("tests/gats/ready.pic", "br") as f:
    words_rus = pc.load( f)

In [ ]:
res_df = pd.DataFrame(columns=["w1","w2","w3", "expected", "prdicted1", "prdicted2", "prdicted3", "cosine", "group"])
for key in words_rus:
    for m_l in words_rus[key]:
        m = []
        for i in m_l:
            if i in cutter:
                m.append(cutter[i])
            else:
                m.append("none")
    for m in words_rus[key]:
        if m[0] in dct and m[1] in dct and m[2] in dct and m[3] in dct:
            pred, prob = get_word(dct[m[0]] - dct[m[1]] + dct[m[3]], dct)
            res_df.loc[res_df.shape[0]] = [m[0], m[1], m[3], m[2], pred, prob, key]
            print(m[2], pred)

NameError: ignored

In [ ]:
res_df.to_csv("tests/gats_result_sent2.csv", index = None, sep = ";", encoding = "cp1251")

In [ ]:
res_df["result"] = res_df["expected"]  == res_df["prdicted"] 

KeyError: ignored

In [ ]:
res_df["result"] = res_df["expected"]  == res_df["prdicted"] 
res_df.groupby("group").mean().sort_values(by = "result")

In [ ]:
res_df.groupby("group").mean().sort_values(by = "cosine")

DataError: ignored

In [ ]:
res_df[res_df["group"] == ": family"]

,w1,w2,w3,expected,prdicted1,prdicted2,prdicted3,cosine,group


In [ ]:
dct = res.copy()

NameError: ignored

In [ ]:
res = []
for i in words_rus[': capital-world']:
    res = res + i

In [ ]:
cutter = {}
nf = []
for i in set(res):
    for j in range(len(i)):
        if i[:len(i) - j] in dct1:
            cutter[i] = i[:len(i) - j]
            break
        if len(i) - j < 4:
            nf.append(i)
            break

In [ ]:
res_df2 = pd.DataFrame(columns=["w1","w2","w3", "expected", "prdicted", "cosine", "group"])
key = ': capital-common-countries'
for m_l in words_rus[key]:
    m = []
    for i in m_l:
        if i in cutter:
            m.append(cutter[i])
        else:
            m.append("none")
    # print(m)
    if m[0] in dct and m[1] in dct and m[2] in dct and m[3] in dct:
        print(m)
        pred, prob = get_word(dct[m[1]] - dct[m[0]] + dct[m[2]], dct)
        res_df2.loc[res_df2.shape[0]] = [m[1], m[0], m[2], m[3], pred, prob, key]
        print(m[3], pred)

KeyError: ignored

In [ ]:
res_df2["result"] = res_df2["expected"]  == res_df2["prdicted"] 
res_df2.groupby("group").mean()

,cosine,result
group,,
: capital-common-countries,0.830527,False


In [ ]:
get_word(dct["россия"] - dct["москва"], dct)

('россия', 0.30764344334602356)

In [ ]:
get_word(dct["ирак"] - dct["багдад"], dct)

('ирак', 0.24611571431159973)

In [ ]:
1 - cosine(dct["багдад"],dct["москва"])

0.6590907573699951

In [ ]:
1 - cosine(dct["ирак"],dct["россия"])

0.7229633927345276

In [ ]:
1 - cosine(dct["багдад"],dct["россия"])

0.6978479623794556

In [ ]:
1 - cosine(dct["ирак"],dct["мать"])

0.6306999325752258

In [ ]:
1 - cosine(dct["мать"] - dct["отец"], dct["внучка"] - dct["внук"])

0.7715850472450256

In [ ]:
res_df = pd.DataFrame(columns=["w1","w2","w3", "expected", "prdicted1", "prdicted2", "prdicted3", "cosine1","cosine2","cosine3", "group"])
for key in words_rus:
    for m_l in words_rus[key]:
        m = []
        for i in m_l:
            if i in cutter:
                m.append(cutter[i])
            else:
                m.append("none")
    for m in words_rus[key]:
        if m[0] in dct and m[1] in dct and m[2] in dct and m[3] in dct:
            ddd = get_word_m(dct[m[0]] - dct[m[1]] + dct[m[3]], dct)
            res_df.loc[res_df.shape[0]] = [m[0], m[1], m[3], m[2], *list(ddd.keys()), *list(ddd.values()), key]
            print(m[2], *list(ddd.keys()))

# Metrics

## 3cosavg

In [ ]:
def cos_avg(dct):
    res_df = pd.DataFrame(columns=["w1","w2","w3", "expected", "prdicted", "cosine", "group"])
    for key in words_rus:
        for m_l in words_rus[key]:
            m = []
            for i in m_l:
                if i in cutter:
                    m.append(cutter[i])
                else:
                    m.append("none")
        for m in words_rus[key]:
            if m[0] in dct and m[1] in dct and m[2] in dct and m[3] in dct:
                pred, prob = get_word(dct[m[0]] - dct[m[1]] + dct[m[3]], dct, nin = [m[0], m[1], m[3]])
                res_df.loc[res_df.shape[0]] = [m[0], m[1], m[3], m[2], pred, prob, key]
                # print(m[2], pred)
    return res_df

In [ ]:
%%time
res_df1 = cos_avg(dct1)
res_df2 = cos_avg(dct2)

CPU times: user 1h 13min 40s, sys: 1min 7s, total: 1h 14min 47s
Wall time: 1h 13min 36s


In [ ]:
res_df1.to_csv("res_df1_avg.csv", encoding = "cp1251", sep = ";")
res_df2.to_csv("res_df2_avg.csv", encoding = "cp1251", sep = ";")

In [ ]:
res_df1 = pd.read_csv("res_df1_avg.csv", encoding = "cp1251", sep = ";")
res_df2 = pd.read_csv("res_df2_avg.csv", encoding = "cp1251", sep = ";")

In [ ]:
res_df1

,Unnamed: 0,w1,w2,w3,expected,prdicted,cosine,group,result
0,0,алжир,алжир,ирак,багдад,багдад,0.840113,: capital-world,True
1,1,тунис,тунис,алжир,алжир,арабов,0.817085,: capital-world,False
2,2,тунис,тунис,ирак,багдад,багдад,0.840113,: capital-world,True
3,3,мальчик,девочка,сестра,брат,брата,0.915870,: family,False
4,4,мальчик,девочка,сестры,братья,брата,0.922371,: family,False
...,...,...,...,...,...,...,...,...,...
1014,1014,китай,китайский,русский,россия,русским,0.917712,: gram6-nationality-adjective,False
1015,1015,египет,египетский,русский,россия,русская,0.891161,: gram6-nationality-adjective,False
1016,1016,египет,египетский,китайский,китай,китай,0.928849,: gram6-nationality-adjective,True
1017,1017,россия,русский,китайский,китай,китайская,0.870841,: gram6-nationality-adjective,False


In [ ]:
res_df1["result"] = res_df1["expected"]  == res_df1["prdicted"] 
count_1 = res_df1.groupby("group").count()
res_sort_1 = res_df1.groupby("group").mean().sort_values(by = "result")
res_sort_1["count"] = count_1["w1"]
res_sort_1 = add_names(res_sort_1)
res_sort_1.index = res_sort_1["type"]
res_sort_1[["cosine", "result", "count"]]

,cosine,result,count
type,,,
Opposite,0.793130,0.000000,20
Comparative,0.845331,0.206522,552
Adjective adverb,0.857969,0.222222,306
Nationality adjective,0.899240,0.333333,6
Family,0.917592,0.659091,132
Capital-countries,0.832437,0.666667,3


In [ ]:
res_df2["result"] = res_df2["expected"]  == res_df2["prdicted"] 
count_2 = res_df2.groupby("group").count()
res_sort_2 = res_df2.groupby("group").mean().sort_values(by = "result")
res_sort_2["count"] = count_2["w1"]
res_sort_2 = add_names(res_sort_2)
res_sort_2.index = res_sort_2["type"]
res_sort_2[["cosine", "result", "count"]]

,cosine,result,count
type,,,
Capital-countries,0.892919,0.000000,3
Opposite,0.792315,0.000000,2
Comparative,0.841756,0.266082,342
Adjective adverb,0.856142,0.271930,342
Family,0.888865,0.309524,42
Nationality adjective,0.906793,0.333333,12


## 3cosadd

In [ ]:
def cos_add(dct):
    res = pd.DataFrame(columns=["w1","p1","w2","p2","prob", "key","count"])
    for key in words_rus:
        res_df = pd.DataFrame(columns=["w1","w2","w3", "expected", "group"])
        for m_l in words_rus[key]:
            m = []
            for i in m_l:
                if i in cutter:
                    m.append(cutter[i])
                else:
                    m.append("none")
        for m in words_rus[key]:
            if m[0] in dct and m[1] in dct and m[2] in dct and m[3] in dct:
                res_df.loc[res_df.shape[0]] = [dct[m[0]].numpy(), 
                                               dct[m[1]].numpy(), 
                                               dct[m[3]].numpy(), 
                                               dct[m[2]].numpy(),
                                               key]
                # print(m[2], pred)
            
            
        w1,p1 = get_word(res_df["w1"].mean() - res_df["w2"].mean() + res_df["w3"].mean(), dct)
        w2,p2 = get_word(res_df["expected"].mean(), dct)
        prob = 1 - cosine(res_df["w1"].mean() - res_df["w2"].mean() + res_df["w3"].mean(), res_df["expected"].mean())
        res.loc[res.shape[0]] = [w1,p1,w2,p2, prob, key, res_df.shape[0]]
    return res

In [ ]:
%%time
res_df1_add = cos_add(dct1)
res_df2_add = cos_add(dct2)

CPU times: user 1min 8s, sys: 1.05 s, total: 1min 9s
Wall time: 1min 8s


In [ ]:
res_df1_add.index = res_df1_add["key"]
add_names(res_df1_add)
res_df1_add.index = res_df1_add["type"]
res_df1_add[["w1", "p1", "w2", "p2", "prob", "count"]]

,w1,p1,w2,p2,prob,count
type,,,,,,
Capital-countries,ирак,0.972100,багдад,0.973289,0.921754,3
Family,отец,0.950429,отец,0.950429,1.000000,132
Adjective adverb,быстрой,0.904212,быстрой,0.904212,1.000000,306
Opposite,возможно,0.910178,возможно,0.910178,1.000000,20
Comparative,сильный,0.931560,сильный,0.931560,1.000000,552
Nationality adjective,египет,0.910278,египет,0.910278,1.000000,6


In [ ]:
res_df2_add.index = res_df2_add["key"]
add_names(res_df2_add)
res_df2_add.index = res_df2_add["type"]
res_df2_add[["w1", "p1", "w2", "p2", "prob", "count"]]

,w1,p1,w2,p2,prob,count
type,,,,,,
Capital-countries,ирак,0.971707,багдад,0.971969,0.928382,3
Family,отец,0.953302,отец,0.953302,1.000000,42
Adjective adverb,последовательный,0.915702,последовательный,0.915702,1.000000,342
Opposite,известный,0.938481,известный,0.938481,1.000000,2
Comparative,сильный,0.931944,сильный,0.931944,1.000000,342
Nationality adjective,египет,0.907877,египет,0.907877,1.000000,12


## 3cosmul

In [ ]:
def get_word_mul(w1,w2,w3, dct):
    word = "none"
    cs = -100 
    for key in dct:
        e1 = 1 - cosine(dct[w1], dct[key])
        e2 = 1 - cosine(dct[w2], dct[key])
        e3 = 1 - cosine(dct[w3], dct[key])
        if e1 - e2 + e3 > cs and key != w1 and key != w2 and key != w3:
            cs = e1 - e2 + e3
            word = key
    return word, cs

In [ ]:
def cos_mul(dct):
    res_df = pd.DataFrame(columns=["w1","w2","w3", "expected", "prdicted", "cosine", "group"])
    for key in words_rus:
        for m_l in words_rus[key]:
            m = []
            for i in m_l:
                if i in cutter:
                    m.append(cutter[i])
                else:
                    m.append("none")
        for m in words_rus[key]:
            if m[0] in dct and m[1] in dct and m[2] in dct and m[3] in dct:
                pred, prob = get_word_mul(m[0], m[1], m[3], dct)
                res_df.loc[res_df.shape[0]] = [m[0], m[1], m[3], m[2], pred, prob, key]
                # print(m[2], pred)
    return res_df

In [ ]:
%%time
res_df1_mul = cos_mul(dct1)
res_df2_mul = cos_mul(dct2)

CPU times: user 3h 51min 15s, sys: 6.49 s, total: 3h 51min 21s
Wall time: 3h 51min 38s


In [ ]:
res_df1_mul["result"] = res_df1_mul["expected"]  == res_df1_mul["prdicted"] 
count_1 = res_df1_mul.groupby("group").count()
res_df_1_mul = res_df1_mul.groupby("group").mean().sort_values(by = "result")
res_df_1_mul["count"] = count_1["w1"]
res_df_1_mul = add_names(res_df_1_mul)
res_df_1_mul.index = res_df_1_mul["type"]
res_df_1_mul[["cosine", "result", "count"]]

,cosine,result,count
type,,,
Антонимы,0.918517,0.000000,20
Сравнение,0.907858,0.215580,552
Прилагательное - наречие,0.930546,0.222222,306
Национальность - прилагательное,0.968291,0.333333,6
Семья,0.937557,0.643939,132
Столицы - страны,0.832437,0.666667,3


In [ ]:
res_df2_mul["result"] = res_df2_mul["expected"]  == res_df2_mul["prdicted"] 
count_2 = res_df2_mul.groupby("group").count()
res_df_2_mul = res_df2_mul.groupby("group").mean().sort_values(by = "result")
res_df_2_mul["count"] = count_2["w1"]
res_df_2_mul = add_names(res_df_2_mul)
res_df_2_mul.index = res_df_2_mul["type"]
res_df_2_mul[["cosine", "result", "count"]]

,cosine,result,count
type,,,
Столицы - страны,0.892919,0.000000,3
Антонимы,0.894300,0.000000,2
Сравнение,0.896573,0.245614,342
Прилагательное - наречие,0.929608,0.269006,342
Семья,0.923934,0.309524,42
Национальность - прилагательное,0.975569,0.333333,12


In [ ]:
res_df1_mul.to_csv("res_df1_mul.csv", encoding = "cp1251", sep = ";")
res_df2_mul.to_csv("res_df2_mul.csv", encoding = "cp1251", sep = ";")


In [ ]:
res_df_1_mul.to_csv("res_df_1_mul.csv", encoding = "cp1251", sep = ";")
res_df_2_mul.to_csv("res_df_2_mul.csv", encoding = "cp1251", sep = ";")

In [ ]:
df1  = pd.read_csv("res_df_1_mul.csv", encoding = "cp1251", sep = ";")
df2  = pd.read_csv("res_df_2_mul.csv", encoding = "cp1251", sep = ";")

In [ ]:
df1["group"] = df1["type.1"]
df1.index = df1["group"]
# df1 = add_names_rus_to_eng(df1 )

df2["group"] = df2["type.1"]
df2.index = df2["group"]
# df2 = add_names_rus_to_eng(df2)

df1.index = df1["type"]
df2.index = df2["type"]

df1.drop(columns = ["type", "type.1", "group"], inplace = True)
df2.drop(columns = ["type", "type.1", "group"], inplace = True)

In [ ]:
df1

,cosine,result,count
type,,,
Opposite,0.918517,0.000000,20
Comparative,0.907858,0.215580,552
Adjective adverb,0.930546,0.222222,306
Nationality adjective,0.968291,0.333333,6
Family,0.937557,0.643939,132
Capital-countries,0.832437,0.666667,3


In [ ]:
df2

,cosine,result,count
type,,,
Capital-countries,0.892919,0.000000,3
Opposite,0.894300,0.000000,2
Comparative,0.896573,0.245614,342
Adjective adverb,0.929608,0.269006,342
Family,0.923934,0.309524,42
Nationality adjective,0.975569,0.333333,12


## mix table

In [ ]:
ddd1 = pd.DataFrame()

In [ ]:
def get_r1(r1, metric):
    r1["metric"] = metric
    return r1.pivot(columns = ["metric"])

In [ ]:
ddd1 = get_r1(res_sort_1[["cosine", "result", "count"]], "3CosAvg")
ddd1 = ddd1.merge(get_r1(df1, "3CosMul"), on = "type")
r1 = res_df1_add[["prob", "count"]].copy()
r1["cosine"] = r1["prob"]
ddd1 = ddd1.merge(get_r1(r1[["cosine", "count"]], "3CosAdd"), on = "type")
ddd1

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


,cosine,result,count,cosine,result,count,cosine,count
metric,3CosAvg,3CosAvg,3CosAvg,3CosMul,3CosMul,3CosMul,3CosAdd,3CosAdd
type,,,,,,,,
Adjective adverb,0.857969,0.222222,306,0.930546,0.222222,306,1.000000,306
Capital-countries,0.832437,0.666667,3,0.832437,0.666667,3,0.921754,3
Comparative,0.845331,0.206522,552,0.907858,0.215580,552,1.000000,552
Family,0.917592,0.659091,132,0.937557,0.643939,132,1.000000,132
Nationality adjective,0.899240,0.333333,6,0.968291,0.333333,6,1.000000,6
Opposite,0.793130,0.000000,20,0.918517,0.000000,20,1.000000,20


In [ ]:
ddd2 = get_r1(res_sort_2[["cosine", "result", "count"]], "3CosAvg")
ddd2 = ddd2.merge(get_r1(df2, "3CosMul"), on = "type")
r2 = res_df2_add[["prob", "count"]].copy()
r2["cosine"] = r2["prob"]
ddd2 = ddd2.merge(get_r1(r2[["cosine", "count"]], "3CosAdd"), on = "type")
ddd2

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


,cosine,result,count,cosine,result,count,cosine,count
metric,3CosAvg,3CosAvg,3CosAvg,3CosMul,3CosMul,3CosMul,3CosAdd,3CosAdd
type,,,,,,,,
Adjective adverb,0.856142,0.271930,342,0.929608,0.269006,342,1.000000,342
Capital-countries,0.892919,0.000000,3,0.892919,0.000000,3,0.928382,3
Comparative,0.841756,0.266082,342,0.896573,0.245614,342,1.000000,342
Family,0.888865,0.309524,42,0.923934,0.309524,42,1.000000,42
Nationality adjective,0.906793,0.333333,12,0.975569,0.333333,12,1.000000,12
Opposite,0.792315,0.000000,2,0.894300,0.000000,2,1.000000,2


In [ ]:
ddd1[["result", "cosine", "count"]]

result              cosine  ...   count                
metric                  3CosAvg   3CosMul   3CosAvg  ... 3CosAvg 3CosMul 3CosAdd
type                                                 ...                        
Adjective adverb       0.222222  0.222222  0.857969  ...     306     306     306
Capital-countries      0.666667  0.666667  0.832437  ...       3       3       3
Comparative            0.206522  0.215580  0.845331  ...     552     552     552
Family                 0.659091  0.643939  0.917592  ...     132     132     132
Nationality adjective  0.333333  0.333333  0.899240  ...       6       6       6
Opposite               0.000000  0.000000  0.793130  ...      20      20      20

[6 rows x 8 columns]

In [ ]:
    ddd2[["result", "cosine", "count"]]

result              cosine  ...   count                
metric                  3CosAvg   3CosMul   3CosAvg  ... 3CosAvg 3CosMul 3CosAdd
type                                                 ...                        
Adjective adverb       0.271930  0.269006  0.856142  ...     342     342     342
Capital-countries      0.000000  0.000000  0.892919  ...       3       3       3
Comparative            0.266082  0.245614  0.841756  ...     342     342     342
Family                 0.309524  0.309524  0.888865  ...      42      42      42
Nationality adjective  0.333333  0.333333  0.906793  ...      12      12      12
Opposite               0.000000  0.000000  0.792315  ...       2       2       2

[6 rows x 8 columns]